LESSON_8 (FINAL PROJECT)

In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

# Для работы с матрицами
from scipy.sparse import csr_matrix

# Матричная факторизация
from implicit import als

# Модель второго уровня
from xgboost import XGBClassifier
from sklearn.model_selection import GridSearchCV

import os, sys
module_path = os.path.abspath(os.path.join(os.pardir))
if module_path not in sys.path:
    sys.path.append(module_path)

# Написанные нами функции
from src.metrics import precision_at_k, recall_at_k
from src.recommenders import MainRecommender
from src.utils import prefilter_items

In [3]:
data = pd.read_csv('data/retail_train_sample.csv')
item_features = pd.read_csv('data/product.csv')
user_features = pd.read_csv('data/hh_demographic.csv')

# column processing
item_features.columns = [col.lower() for col in item_features.columns]
user_features.columns = [col.lower() for col in user_features.columns]

item_features.rename(columns={'product_id': 'item_id'}, inplace=True)
user_features.rename(columns={'household_key': 'user_id'}, inplace=True)


# Важна схема обучения и валидации!
# -- давние покупки -- | -- 6 недель -- | -- 3 недель -- 
# подобрать размер 2-ого датасета (6 недель) --> learning curve (зависимость метрики recall@k от размера датасета)
val_lvl_1_size_weeks = 6
val_lvl_2_size_weeks = 3

data_train_lvl_1 = data[data['week_no'] < data['week_no'].max() - (val_lvl_1_size_weeks + val_lvl_2_size_weeks)]
data_val_lvl_1 = data[(data['week_no'] >= data['week_no'].max() - (val_lvl_1_size_weeks + val_lvl_2_size_weeks)) &
                      (data['week_no'] < data['week_no'].max() - (val_lvl_2_size_weeks))]

data_train_lvl_2 = data_val_lvl_1.copy()  # Для наглядности. Далее мы добавим изменения, и они будут отличаться
data_val_lvl_2 = data[data['week_no'] >= data['week_no'].max() - val_lvl_2_size_weeks]

data_train_lvl_1.head(2)

,Unnamed: 0,user_id,basket_id,day,item_id,quantity,sales_value,store_id,retail_disc,trans_time,week_no,coupon_disc,coupon_match_disc
0,1832874,1078,35573861879,524,1082185,1,0.56,375,0.0,1440,76,0.0,0.0
1,402281,324,29170411703,165,7168774,2,6.98,367,0.0,1115,24,0.0,0.0


In [4]:
n_items_before = data_train_lvl_1['item_id'].nunique()

data_train_lvl_1 = prefilter_items(data_train_lvl_1, item_features=item_features, take_n_popular=5000)

n_items_after = data_train_lvl_1['item_id'].nunique()
print('Decreased # items from {} to {}'.format(n_items_before, n_items_after))

Decreased # items from 33411 to 5001


In [5]:
recommender = MainRecommender(data_train_lvl_1)

  0%|          | 0/15 [00:00<?, ?it/s]

  0%|          | 0/5001 [00:00<?, ?it/s]

In [6]:
result_lvl_1 = data_val_lvl_1.groupby('user_id')['item_id'].unique().reset_index()
result_lvl_1.columns=['user_id', 'actual']
result_lvl_1.head(2)

,user_id,actual
0,1,"[1098248, 1017299, 1077430, 938004, 1082185, 8..."
1,2,"[830127, 7442008, 899624, 944568]"


In [7]:
def do_recommend(model, func, n, k):
    
    func_dic = {
        'get_als_recommendations': model.get_als_recommendations,
        'get_own_recommendations': model.get_own_recommendations,
        'get_similar_items_recommendation': model.get_similar_items_recommendation,
        'get_similar_users_recommendation': model.get_similar_users_recommendation
    }
    
    try:
        result = func_dic[func](n, k)
    
    except (IndexError, ValueError) as e:
        
        result = recommender._extend_with_top_popular([], k)
    
    return result

In [8]:
%%time 

k = 100
result_lvl_1['als'] = result_lvl_1['user_id'].apply(lambda x: 
                                                    do_recommend(recommender, 'get_als_recommendations', x, k))
result_lvl_1.head(2)

CPU times: user 20.1 s, sys: 56.6 s, total: 1min 16s
Wall time: 10.7 s


,user_id,actual,als
0,1,"[1098248, 1017299, 1077430, 938004, 1082185, 8...","[1029743, 995242, 844179, 914190, 1082185, 108..."
1,2,"[830127, 7442008, 899624, 944568]","[951590, 1082185, 5569230, 1029743, 962229, 10..."


In [9]:
result_lvl_1.apply(lambda row: precision_at_k(row['als'], row['actual'], k=5), axis=1).mean()

0.04874213836477988

In [10]:
users_lvl_2 = pd.DataFrame(data_train_lvl_2['user_id'].unique())
users_lvl_2.columns = ['user_id']

users_lvl_2.head(2)

,user_id
0,1501
1,1633


In [11]:
# только warm start
train_users = data_train_lvl_1['user_id'].unique()
users_lvl_2 = users_lvl_2[users_lvl_2['user_id'].isin(train_users)]

users_lvl_2['candidates'] = users_lvl_2['user_id'].apply(lambda x: recommender.get_own_recommendations(x, N=50))
users_lvl_2.head(4)

,user_id,candidates
0,1501,"[1082185, 1029743, 995242, 1106523, 981760, 11..."
1,1633,"[1082185, 1029743, 995242, 1106523, 981760, 11..."
2,336,"[1082185, 1029743, 995242, 1106523, 981760, 11..."
3,2195,"[1082185, 1029743, 995242, 1106523, 981760, 11..."


In [12]:
s = users_lvl_2.apply(lambda x: pd.Series(x['candidates']), axis=1).stack().reset_index(level=1, drop=True)
s.name = 'item_id'

users_lvl_2 = users_lvl_2.drop('candidates', axis=1).join(s)
users_lvl_2['flag'] = 1

users_lvl_2.head(4)

,user_id,item_id,flag
0,1501,1082185,1
0,1501,1029743,1
0,1501,995242,1
0,1501,1106523,1


In [13]:
targets_lvl_2 = data_train_lvl_2.copy()
targets_lvl_2['target'] = 1  # тут только покупки 

targets_lvl_2 = users_lvl_2.merge(targets_lvl_2, on=['user_id', 'item_id'], how='left')

targets_lvl_2['target'].fillna(0, inplace= True)
targets_lvl_2.drop('flag', axis=1, inplace=True)

In [14]:
targets_lvl_2['target'].mean()

0.015964099168689766

In [15]:
item_features.head(2)

,item_id,manufacturer,department,brand,commodity_desc,sub_commodity_desc,curr_size_of_product
0,25671,2,GROCERY,National,FRZN ICE,ICE - CRUSHED/CUBED,22 LB
1,26081,2,MISC. TRANS.,National,NO COMMODITY DESCRIPTION,NO SUBCOMMODITY DESCRIPTION,


In [16]:
user_features.head(2)

,age_desc,marital_status_code,income_desc,homeowner_desc,hh_comp_desc,household_size_desc,kid_category_desc,user_id
0,65+,A,35-49K,Homeowner,2 Adults No Kids,2,None/Unknown,1
1,45-54,A,50-74K,Homeowner,2 Adults No Kids,2,None/Unknown,7


In [17]:
targets_lvl_2 = targets_lvl_2.merge(item_features, on='item_id', how='left')
targets_lvl_2 = targets_lvl_2.merge(user_features, on='user_id', how='left')

targets_lvl_2.head(2)

,user_id,item_id,Unnamed: 0,basket_id,day,quantity,sales_value,store_id,retail_disc,trans_time,...,commodity_desc,sub_commodity_desc,curr_size_of_product,age_desc,marital_status_code,income_desc,homeowner_desc,hh_comp_desc,household_size_desc,kid_category_desc
0,1501,1082185,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,TROPICAL FRUIT,BANANAS,40 LB,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1501,1029743,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,FLUID MILK PRODUCTS,FLUID MILK WHITE ONLY,1 GA,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [18]:
# добавим средний чек по каждому пользователю

basket_sum = targets_lvl_2.groupby(['user_id', 'basket_id'], as_index=False).agg({'sales_value': 'sum'})
basket_mean = basket_sum.groupby(['user_id'], as_index=False).agg({'sales_value': 'mean'}).\
            rename(columns={'sales_value':'basket_mean'})

targets_lvl_2 = targets_lvl_2.merge(basket_mean, on='user_id', how='left')

targets_lvl_2.head(2)

,user_id,item_id,Unnamed: 0,basket_id,day,quantity,sales_value,store_id,retail_disc,trans_time,...,sub_commodity_desc,curr_size_of_product,age_desc,marital_status_code,income_desc,homeowner_desc,hh_comp_desc,household_size_desc,kid_category_desc,basket_mean
0,1501,1082185,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,BANANAS,40 LB,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.99
1,1501,1029743,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,FLUID MILK WHITE ONLY,1 GA,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.99


In [19]:
# кол-во покупок в каждой категрии по пользователям

user_cat_sale_count = targets_lvl_2.groupby(['user_id', 'commodity_desc'], as_index=True).\
                        agg({'commodity_desc': 'count'}).\
                        rename(columns={'commodity_desc':'commodity_count'}).reset_index()

targets_lvl_2 = targets_lvl_2.merge(user_cat_sale_count, on=['user_id', 'commodity_desc'], how='left')
targets_lvl_2.head(2)

,user_id,item_id,Unnamed: 0,basket_id,day,quantity,sales_value,store_id,retail_disc,trans_time,...,curr_size_of_product,age_desc,marital_status_code,income_desc,homeowner_desc,hh_comp_desc,household_size_desc,kid_category_desc,basket_mean,commodity_count
0,1501,1082185,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,40 LB,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.99,1
1,1501,1029743,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,1 GA,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.99,9


In [20]:
# кол-во покупок категории в неделю

commodity_week_count = targets_lvl_2.groupby(['commodity_desc', 'week_no'], as_index=True).\
                        agg({'commodity_desc': 'count'}).\
                        rename(columns={'commodity_desc':'commodity_week_count'}).\
                        groupby(['commodity_desc']).agg({'commodity_week_count': 'mean'}).reset_index()

targets_lvl_2 = targets_lvl_2.merge(commodity_week_count, on='commodity_desc', how='left')
targets_lvl_2.head(2)

,user_id,item_id,Unnamed: 0,basket_id,day,quantity,sales_value,store_id,retail_disc,trans_time,...,age_desc,marital_status_code,income_desc,homeowner_desc,hh_comp_desc,household_size_desc,kid_category_desc,basket_mean,commodity_count,commodity_week_count
0,1501,1082185,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.99,1,33.833333
1,1501,1029743,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.99,9,66.666667


In [21]:
# стоимость товара к средней стоимости по категории

commodity_mean_price = targets_lvl_2.groupby(['commodity_desc', 'item_id'], as_index=True).\
                        agg({'sales_value': 'mean'}).\
                        rename(columns={'sales_value':'item_price'}).\
                        groupby(['commodity_desc']).agg({'item_price': 'mean'}).reset_index()

targets_lvl_2 = targets_lvl_2.merge(commodity_mean_price, on='commodity_desc', how='left')
targets_lvl_2['price_to_cat'] = targets_lvl_2[['sales_value', 'item_price']].\
                    apply(lambda row: row['sales_value']/row['item_price'], axis=1)
targets_lvl_2.head(2)

,user_id,item_id,Unnamed: 0,basket_id,day,quantity,sales_value,store_id,retail_disc,trans_time,...,income_desc,homeowner_desc,hh_comp_desc,household_size_desc,kid_category_desc,basket_mean,commodity_count,commodity_week_count,item_price,price_to_cat
0,1501,1082185,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,1.99,1,33.833333,1.087882,NaN
1,1501,1029743,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,1.99,9,66.666667,2.406886,NaN


In [22]:
X_train = targets_lvl_2.drop('target', axis=1)
y_train = targets_lvl_2[['target']]

In [23]:
item_features.columns, user_features.columns

(Index(['item_id', 'manufacturer', 'department', 'brand', 'commodity_desc',
        'sub_commodity_desc', 'curr_size_of_product'],
       dtype='object'),
 Index(['age_desc', 'marital_status_code', 'income_desc', 'homeowner_desc',
        'hh_comp_desc', 'household_size_desc', 'kid_category_desc', 'user_id'],
       dtype='object'))

In [24]:
cat_feats = ['manufacturer', 'department', 'brand', 'commodity_desc',
       'sub_commodity_desc', 'curr_size_of_product', 'age_desc', 'marital_status_code', 'income_desc', 'homeowner_desc',
        'hh_comp_desc', 'household_size_desc', 'kid_category_desc']

X_train[cat_feats] = X_train[cat_feats].astype('category')

In [25]:
X_train = pd.get_dummies(X_train, columns=cat_feats, drop_first=True)

In [26]:
xgb_model = XGBClassifier(random_state=42)

In [27]:
parameters = {
    'n_estimators': [50, 100, 200, 300],
    'max_depth': [2, 3, 4],
    'max_leaves': [2, 3],
    'objective': ['binary:logistic', 'rank:pairwise']
}
grid_search = GridSearchCV(estimator = xgb_model,
                           param_grid = parameters,
                           scoring = 'accuracy',
                           cv = 10,
                           n_jobs = -1)

In [28]:
grid_search = grid_search.fit(X_train, y_train)

In [29]:
data_test = pd.read_csv('data/retail_test1.csv')
data_test = prefilter_items(data=data_test, take_n_popular=5000, item_features=item_features)
data_test.head(5)

,user_id,basket_id,day,item_id,quantity,sales_value,store_id,retail_disc,trans_time,week_no,coupon_disc,coupon_match_disc,price
0,1340,41652823310,664,912987,1,8.49,446,0.0,52,96,0.0,0.0,8.49
1,588,41652838477,664,1024426,1,6.29,388,0.0,8,96,0.0,0.0,6.29
2,2070,41652857291,664,995242,5,9.10,311,-0.6,46,96,0.0,0.0,1.82
3,1602,41665647035,664,999999,1,7.99,334,0.0,1741,96,0.0,0.0,7.99
5,1602,41665647035,664,999999,1,3.29,334,-1.2,1741,96,0.0,0.0,3.29


In [31]:
data_test = data_test[data_test['user_id'].isin(data['user_id'].unique())]
# data_test = data_test[data_test['item_id'].isin(data_train['item_id'].unique())]
data_test

,user_id,basket_id,day,item_id,quantity,sales_value,store_id,retail_disc,trans_time,week_no,coupon_disc,coupon_match_disc,price
0,1340,41652823310,664,912987,1,8.49,446,0.0,52,96,0.0,0.0,8.49
1,588,41652838477,664,1024426,1,6.29,388,0.0,8,96,0.0,0.0,6.29
2,2070,41652857291,664,995242,5,9.10,311,-0.6,46,96,0.0,0.0,1.82
3,1602,41665647035,664,999999,1,7.99,334,0.0,1741,96,0.0,0.0,7.99
5,1602,41665647035,664,999999,1,3.29,334,-1.2,1741,96,0.0,0.0,3.29
...,...,...,...,...,...,...,...,...,...,...,...,...,...
88728,98,41944918649,684,1128647,1,1.00,421,0.0,1702,98,0.0,0.0,1.00
88729,98,41944918665,684,945779,2,2.00,421,0.0,1705,98,0.0,0.0,1.00
88730,98,41944918665,684,993617,2,2.00,421,0.0,1705,98,0.0,0.0,1.00
88731,98,41944918665,684,1128647,2,2.00,421,0.0,1705,98,0.0,0.0,1.00


In [32]:
result_test = data_test.groupby('user_id')['item_id'].unique().reset_index()
result_test.columns=['user_id', 'actual']
result_test.head(2)

,user_id,actual
0,1,"[999999, 883616, 931136, 940947, 958046, 96155..."
1,2,"[820291, 826784, 999999, 866211, 870608, 88502..."


In [33]:
%%time 

k = 50
result_test['als'] = result_test['user_id'].apply(lambda x: 
                                    recommender.get_als_recommendations(x, k))

CPU times: user 12.6 s, sys: 33.2 s, total: 45.8 s
Wall time: 6.2 s


In [35]:
precision_k = result_test['user_id'].apply(lambda x: 
                                    precision_at_k(result_test.loc[result_lvl_1['user_id']==x, 
                                                                          'als'].squeeze(), 
                                                   result_test.loc[result_lvl_1['user_id']==x, 
                                                                          'actual'].squeeze(), 
                                                   k=5)).mean()

precision_k

0.2356377551020408